# Extract Data

In [1]:
import json
import pandas as pd

with open('../data/concatenated_data.txt', 'r', encoding='latin-1') as file:
    lines = file.readlines()

latitudes = []
longitudes = []
altitudes = []

for idx, line in enumerate(lines):
    try:
        data = json.loads(line)
        if not isinstance(data, dict):
            continue
    except Exception as e:
        continue

    if 'Location' in data:
        data_location = data['Location']
        latitudes.append(data_location['Latitude'])
        longitudes.append(data_location['Longtitude'])
        altitudes.append(data_location['Altitude'])

df_locations = pd.DataFrame({"Latitude": latitudes, "Longitude": longitudes, "Altitude": altitudes})

df_locations.to_csv("../data/locations.csv", index=False)

/tmp/ipykernel_12093/130720007.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_locations = pd.read_csv("../data/locations.csv")